In [1]:
import os
import tensorflow as tf
print(tf.version.VERSION)
if tf.__version__.startswith('1.15'):
    # This prevents some errors that otherwise occur when converting the model with TF 1.15...
    tf.enable_eager_execution() # Only if TF is version 1.15
from keras_vggface_TF.vggfaceTF import VGGFace
print(os.getcwd())
pretrained_model = VGGFace(model='resnet50', include_top=True, input_shape=(224, 224, 3), pooling='avg')  # pooling: None, avg or max
#pretrained_model.summary()
#pretrained_model.save('VGGmodelTF')


2.2.0
/home/base/Documents/Git/Projekte/Quantization
I Am  none


Using TensorFlow backend.


In [2]:
from pathlib import Path
from keras_vggface_TF.utils import preprocess_input

base_dir_home=Path('/home/base/Documents/Git/Projekte/CelebFaceMatcher/Finetunedata/')
train_dir_home= base_dir_home / 'Train'
test_dir_home= base_dir_home / 'Test'


#Create Data generatro fwith preprocessed images

def Vgg_Preprocess(img):
#     /img = img.astype('float32')
    #img = expand_dims(img, axis=0)
    img = preprocess_input(img, version=2) 
    return img

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=Vgg_Preprocess)

# train_generator = train_datagen.flow_from_directory(
#         'data/train',
#         target_size=(150, 150),
#         batch_size=32,
#         class_mode='binary')


# Configure the dataset for performance
# Use buffered prefetching to load images from disk without having I/O become blocking. To learn more about this method see the data performance guide. https://www.tensorflow.org/guide/data_performance

# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Create dataset from directory

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = train_datagen.flow_from_directory(train_dir_home,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             target_size=IMG_SIZE,
                                             seed=42,
                                             )

validation_dataset = train_datagen.flow_from_directory(test_dir_home,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             target_size=IMG_SIZE,
                                             seed=42,
                                             )


Found 9021 images belonging to 8555 classes.
Found 538 images belonging to 480 classes.


In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import tensorflow.keras.optimizers



#Set which quantisatiomn option you choose. qunatize_model sets everything to default and quantizes the whole model
# Set the pretrained model to quantization state, adding fake nodes
q_aware_model = tfmot.quantization.keras.quantize_model(pretrained_model)

# `quantize_model` requires a recompile before training.
# q_aware_model.compile(
#     optimizer= tf.keras.optimizers.Adam,
#     loss=tf.keras.losses.binary_crossentropy,
#     metrics= tf.keras.metrics.Accuracy
# )
q_aware_model.compile(
    optimizer= 'Adam',
    loss=tf.keras.losses.binary_crossentropy,
    metrics= ['accuracy']
)
#Train or finetune your model now
initial_epochs = 10


history = q_aware_model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)


# Now finetune

# # Fine-tune from this layer onwards
# fine_tune_at = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in quantize_model.layers[:fine_tune_at]:
#   layer.trainable =  False


Epoch 1/10


In [ ]:
import cv2
import numpy as np

def representative_dataset_gen():
  for _ in range(10):
    pfad='/home/base/Documents/Git/Projekte/CelebFaceMatcher/All_croped_images/Aaron Paul/000001.jpg'
    img=cv2.imread(pfad)
    img = np.expand_dims(img,0).astype(np.float32) 
    yield [img]

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
    
converter.representative_dataset = representative_dataset_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.experimental_new_converter = True

converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.int8 # or tf.uint8
converter.inference_output_type = tf.int8 # or tf.uint8
quantized_tflite_model = converter.convert()


if tf.__version__.startswith('1.'):
    open("QAT1153_all_int8.tflite", "wb").write(quantized_tflite_model)
if tf.__version__.startswith('2.'):
    with open("QAT220_all_int8.tflite", 'wb') as f:
        f.write(quantized_tflite_model)# mit 220 vs 2_2_0 Ich hatte im modelcode dtype int32 und int8 eingefüght. Jetzt wieder draußen
print('I am outahere')